### Modeling/Feature Analysis

In [1]:
import time
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import seaborn as sns
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn import tree
from sklearn.ensemble import RandomForestRegressor
from sklearn import svm

In [2]:
sales = pd.read_csv("prepared.csv")

In [3]:
sales = pd.concat([sales,pd.get_dummies(sales['category'])],axis=1)
sales = pd.concat([sales,pd.get_dummies(sales['day'])],axis=1)
sales = pd.concat([sales,pd.get_dummies(sales['customer_state_x'])],axis=1)
sales =sales.drop(['Unnamed: 0','customer_state_x','customer_zip_code_prefix','customer_city_x','customer_city_y','customer_state_y'],axis=1)
sales =sales.drop(['category','day'],axis=1)
sales.describe()

,Time_to_customer,price,quantity,Year,Week_Num,month,agro_industry_and_commerce,air_conditioning,art,audio,...,PR,RJ,RN,RO,RR,RS,SC,SE,SP,TO
count,345967.000000,350268.000000,350268.000000,350268.000000,350268.000000,350268.000000,350268.000000,350268.000000,350268.000000,350268.000000,...,350268.000000,350268.000000,350268.000000,350268.000000,350268.000000,350268.000000,350268.000000,350268.000000,350268.000000,350268.000000
mean,12.736683,109.955560,1.934796,2017.523965,24.647199,6.092092,0.003158,0.000260,0.004499,0.005747,...,0.032726,0.221790,0.001342,0.000731,0.000071,0.046225,0.035199,0.001924,0.374111,0.001039
std,10.026212,106.839584,2.176580,0.501457,14.631946,3.357927,0.056104,0.016116,0.066927,0.075591,...,0.177920,0.415451,0.036606,0.027025,0.008448,0.209971,0.184282,0.043824,0.483893,0.032220
min,0.000000,6.900000,1.000000,2016.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7.000000,53.900000,1.000000,2017.000000,12.000000,3.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,10.000000,89.900000,1.000000,2018.000000,24.000000,6.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,16.000000,129.990000,2.000000,2018.000000,36.000000,9.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000
max,208.000000,1997.000000,25.000000,2018.000000,52.000000,12.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [4]:
sales =sales.dropna()
Y = sales['quantity']
X = sales.drop(['quantity'],axis=1)
X_Train, X_Test, Y_Train, Y_Test = train_test_split(X, Y, test_size = 0.30, 
                                                    random_state = 101)

### Feature Selction

In [5]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from IPython.core.pylabtools import figsize

In [6]:
sales

,Time_to_customer,price,quantity,Year,Week_Num,month,agro_industry_and_commerce,air_conditioning,art,audio,...,PR,RJ,RN,RO,RR,RS,SC,SE,SP,TO
0,13.0,118.7,1,2018,30,7,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,13.0,118.7,1,2018,30,7,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5.0,116.9,1,2018,32,8,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,5.0,116.9,1,2018,32,8,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,5.0,116.9,1,2018,32,8,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350263,6.0,187.0,1,2018,23,6,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
350264,6.0,187.0,1,2018,23,6,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
350265,6.0,187.0,1,2018,23,6,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
350266,8.0,188.0,1,2018,19,5,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
#apply SelectKBest class to extract top 10 best features
bestfeatures = SelectKBest(score_func=chi2, k=10)
fit = bestfeatures.fit(X,Y)
dfscores = pd.DataFrame(fit.scores_)
dfcolumns = pd.DataFrame(X.columns)

In [8]:
#concat two dataframes for better visualization 
featureScores = pd.concat([dfcolumns,dfscores],axis=1)
featureScores.columns = ['Columns','Scores']  #naming the dataframe columns
print(featureScores.nlargest(10,'Scores')) 

             Columns         Scores
1              price  386360.780916
0   Time_to_customer   60180.959961
3           Week_Num   52657.479482
31      garden_tools   17176.837126
28   furniture_decor   16173.345968
52     watches_gifts   10629.681784
4              month   10423.094595
53            Friday    8404.520066
58           Tuesday    3307.894315
55          Saturday    2976.526689


### SVM

In [9]:
X_reduced = X[list(featureScores.nlargest(10,'Scores')['Columns'])]
X_Train2, X_Test2, Y_Train2, Y_Test2 = train_test_split(X_reduced, Y, test_size = 0.30,random_state = 101)

In [14]:
from sklearn.svm import SVR
SVR = SVR(kernel = 'poly', C = 10)

In [15]:
sc = StandardScaler()
X_Train2 = sc.fit_transform(X_Train2.astype(np.float))
X_Test2 = sc.transform(X_Test2.astype(np.float))

In [16]:
#fit model
# svr = SVR.fit(X_Train2, Y_Train2.values.ravel())
# y_train_pred = svr.predict(X_Train2)
# y_test_pred = svr.predict(X_Test2)

In [17]:
# print(f'svr train score is {svr.score(X_Train2, Y_Train2)}, svr test score is {svr.score(X_Test2, Y_Test2)}')

### Poly regression

In [19]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures

In [23]:
poly = PolynomialFeatures(degree = 5)
X_poly = poly.fit_transform(X_reduced)

X_train3,X_test3,y_train3,y_test3 = train_test_split(X_poly,Y, test_size = 0.3, random_state = 101)


In [24]:
#fit model
poly_lr = LinearRegression().fit(X_train3, y_train3)
y_train_pred = poly_lr.predict(X_train3)
y_test_pred = poly_lr.predict(X_test3)

In [25]:
print(f'poly score is {poly_lr.score(X_train3, y_train3)}, poly test score is {poly_lr.score(X_test3, y_test3)}')

poly score is 0.4533116618762604, poly test score is 0.4523070995839411


### Decision Tree

In [50]:
from sklearn.tree import DecisionTreeRegressor

In [51]:
dt = DecisionTreeRegressor(random_state = 0)

In [48]:
X_train4, X_test4, y_train4, y_test4 = train_test_split(X_reduced, Y, test_size = 0.30, random_state = 101 )

In [52]:
#fit model
dt = dt.fit(X_train4,y_train4.values.ravel())
y_train_pred = dt.predict(X_train4)
y_test_pred = dt.predict(X_test4)

In [53]:
print(f'dt score is {dt.score(X_train4, y_train4)}, dt test score is {dt.score(X_test4, y_test4)}')

dt score is 0.9516451061394917, dt test score is 0.9459651888350719
